In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName('mllr').getOrCreate()

这个数据格式比较特殊,libsvm格式

In [ ]:
training = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [4]:
training.show()

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
| -7.896274316726144|(10,[0,1,2,3,4,5,...|
| -8.464803554195287|(10,[0,1,2,3,4,5,...|
| 2.1214592666251364|(10,[0,1,2,3,4,5,...|
| 1.0720117616524107|(10,[0,1,2,3,4,5,...|
|-13.772441561702871|(10,[0,1,2,3,4,5,...|
| -5.082010756207233|(10,[0,1,2,3,4,5,...|
|  7.887786536531237|(10,[0,1,2,3,4,5,...|
| 14.323146365332388|(10,[0,1,2,3,4,5,...|
|-20.057482615789212|(10,[0,1,2,3,4,5,...|
|-0.8995693247765151|(10,[0,1,2,3,4,5,...|
| -19.16829262296376|(10,[0,1,2,3,4,5,...|
|  5.601801561245534|(10,[0,1,2,3,4,5,...|
|-3.2256352187273354|(10,[0,1,2,3,4,5,...|
| 1.5299675726687754|(10,[0,1,2,3,4,5,...|
| -0.250102447941961|(10,[0,1,2,3,4,5,...|
+----------

创建线性回归的实例

In [5]:
lr = LinearRegression(featuresCol='features', labelCol='label',
                     predictionCol='prediction')

In [6]:
lrModel = lr.fit(training)

In [7]:
lrModel.coefficients

DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [8]:
lrModel.intercept

0.14228558260358093

In [9]:
summary = lrModel.summary

In [10]:
summary.r2

0.027839179518600154

这个例子只是简单的使用了库，并没有把数据分成训练数据和测试数据，下面进行分组的练习

In [12]:
allDatas = spark.read.format('libsvm').load('sample_linear_regression_data.txt')

In [14]:
splitDatas = allDatas.randomSplit([0.7, 0.3])

In [16]:
trainingData, testData = splitDatas

In [18]:
trainingData.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                354|
|   mean| 0.2401417138052728|
| stddev| 10.282741341762986|
|    min|-28.571478869743427|
|    max| 27.111027963108548|
+-------+-------------------+



In [19]:
testData.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                147|
|   mean|  0.297218593419208|
| stddev| 10.437240168494446|
|    min|-28.046018037776633|
|    max|  27.78383192005107|
+-------+-------------------+



In [21]:
correctModel = lr.fit(trainingData)

In [22]:
res = correctModel.evaluate(testData)

In [23]:
res.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| -27.55178910831191|
| -22.05270239874738|
|-26.401645457410083|
|-20.019691033562207|
|-18.227803942548956|
|-20.285790964273712|
|  -16.1776859418418|
| -16.96932121429246|
|-15.274429664127156|
|-16.134882082306625|
|-13.091971075030157|
|-12.325244594857645|
|  -9.65816961890835|
|-13.260477528504769|
|-13.578680418817866|
|-10.607276679333486|
|-14.782877765915973|
|  -7.49845607084201|
|-14.101277319005048|
| -9.037507472647539|
+-------------------+
only showing top 20 rows



需要增加一些基础知识，模型和重要参数等

In [24]:
unlabeled_data = testData.select('features')

In [25]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
|(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows



In [26]:
preres = correctModel.transform(unlabeled_data)

In [27]:
preres.show()

+--------------------+-------------------+
|            features|         prediction|
+--------------------+-------------------+
|(10,[0,1,2,3,4,5,...|-0.4942289294647209|
|(10,[0,1,2,3,4,5,...|-1.4581816915755914|
|(10,[0,1,2,3,4,5,...| 3.4518195212140093|
|(10,[0,1,2,3,4,5,...|0.13513025928878247|
|(10,[0,1,2,3,4,5,...|-1.6451870955194507|
|(10,[0,1,2,3,4,5,...| 0.5030281746591733|
|(10,[0,1,2,3,4,5,...|-0.8488063223677458|
|(10,[0,1,2,3,4,5,...| 0.8179718630153484|
|(10,[0,1,2,3,4,5,...|-0.5875796634434033|
|(10,[0,1,2,3,4,5,...| 1.8059035732311826|
|(10,[0,1,2,3,4,5,...|-0.7751168201286115|
|(10,[0,1,2,3,4,5,...|-1.0953501810331123|
|(10,[0,1,2,3,4,5,...|-3.1150573803428476|
|(10,[0,1,2,3,4,5,...| 0.7597037431497143|
|(10,[0,1,2,3,4,5,...| 1.0872383412714537|
|(10,[0,1,2,3,4,5,...|-1.5230765329544433|
|(10,[0,1,2,3,4,5,...| 2.9558047695234024|
|(10,[0,1,2,3,4,5,...| -4.142093607046816|
|(10,[0,1,2,3,4,5,...| 2.6694749534645883|
|(10,[0,1,2,3,4,5,...|-1.5401908315956825|
+----------

### 模型评估

通常使用的评估指标

* Mean Absolute Error
* Mean Squared Error
* Root Mean Square Error
* R Squared Values 

需要对 [PPT](https://docs.google.com/presentation/d/1HH-AawBoa6VtuZOOaN3oJNtbfodBpm63j6ddL1yS2PQ/edit#slide=id.g1de074efe1_0_85) 中的内容进一步学习 